## Starting with the API

This tutorial explains how to use the `api` module. The `RadarlyApi` object is defined in this module and is used by `radarly` in order to handle communications with the API. For example, it simplifies the OAuth2 authorization system and can also refresh your access token when it has expired.

In [1]:
from radarly import RadarlyApi
from requests.exceptions import ProxyError
from pprint import pprint

### Initialization

The `RadarlyApi` object must be initialized with at least two arguments : the `client_id` and the `client_secret`. These credentials are given by Linkfluence and must not be shared because they are used in order to authentify each request to our APIs. If you don't have these credentials to start with the client, please contact us to python@linkfluence.com.

In [2]:
credentials = {
    "client_id": ":my_client_id:",
    "client_secret": ":my_client_secret:"
}
api = RadarlyApi(**credentials, scope=['listening', 'social-performance'])
api

<RadarlyAPI.client_id=:my_client_id:>

Some additional parameters can be set directly during the initialization of the API and will be used each time a request is made. For example, you can set  proxies and timeout for your requests. The proxies object must have the same template of the proxies object defined in `requests` module.

In [3]:
proxies = {
    'http': 'http://127.0.0.1:9000',
    'https': 'https://127.0.0.1:9000'
}
timeout = 120

try:
    RadarlyApi(**credentials, proxies=proxies, timeout=timeout)
except ProxyError:
    print('Warning: This will not work without a valid proxy.')

<div class="alert alert-warning">

Given that some requests in the API can take tens of seconds to process, you should consider to not to set a too low value for the `timeout` argument. Otherwise, some requests can be stopped only because the timeout has expired.

<div>

Each optional arguments can be set after initialization.

In [4]:
api.proxies = None
api.timeout = 2

In order not to have to specify the API object in each request, you can set a default API. This API will be used each time you make a request to the ``RadarlyApi``  if no API object is specified.

In [5]:
RadarlyApi.set_default_api(api)
# You can also retrieve the default API
api = RadarlyApi.get_default_api()
api

<RadarlyAPI.client_id=:my_client_id:>

In order to initialize a new API and set it as the default API, yon can use the ``init`` class method. This method uses the same arguments as ``__init__`` method.

In [6]:
api = RadarlyApi.init(**credentials, timeout=200, proxies=None)
api == RadarlyApi.get_default_api()

True

Notice that rather than specifying the ``client_id`` and  ``client_secret`` argument during the initialization of the API, you can set the ``RADARLY_CLIENT_ID`` and ``RADARLY_CLIENT_SECRET`` environment variables. ``RadarlyApi`` will automatically retrieve these variables if ``client_id`` and ``client_secret`` is set to ``None``.

In [7]:
%%bash
export RADARLY_CLIENT_ID=:my_client_id:
export RADARLY_CLIENT_SECRET=:my_client_secret:

In [8]:
!export RADARLY_CLIENT_ID=:my_client_id:
!export RADARLY_CLIENT_SECRET=:my_client_secret:
RadarlyApi.init(scope=['listening', 'social-performance'])

<RadarlyAPI.client_id=:my_client_id:>

### Make some requests

The ``RadarlyApi`` object can now be used in order to make each requests to the Radarly API. The `request` method defined in `RadarlyApi` used the same arguments as the `request` function of the `requests` module, but some behaviours are different. For example, if the pattern `{API_Host}/{version}` is not present in the URL of your request, it will be automatically added at the beginning of your URL.

<div class="alert alert-info">
The root url and the version of the API are class variable of the `RadarlyApi` object.
</div>

In [9]:
print('Host of the API:', api.root_url)
print('Current version of the API:', api.version)

Host of the API: https://radarly.linkfluence.com
Current version of the API: 1.0


You can now perform a request, using any relative path exposed in our API. For example, to get informations about the current user.

In [10]:
current_user = api.request('GET', 'https://radarly.linkfluence.com/1.0/users.json')

As in ``requests`` module, you can use the ``get`` method to perform a request with ``GET``.

In [11]:
current_user = api.get('/users.json')
pprint(current_user, depth=1)

{'account_id': 1,
 'apps': [...],
 'can_create_project': True,
 'connected': 1526635380515,
 'connection_count': 102,
 'created': datetime.datetime(2018, 2, 12, 17, 59, 55, 105000, tzinfo=tzutc()),
 'current_project_id': 3740,
 'email': 'alexandre.aheto@linkfluence.com',
 'engagement': {...},
 'id': 16246,
 'is_disabled': False,
 'is_internal': True,
 'is_manager': True,
 'is_pending': False,
 'is_root': True,
 'level': 'root',
 'locale': 'en_GB',
 'name': 'Alexandre Aheto',
 'picture_id': None,
 'projects': [...],
 'theme': 'dark',
 'timezone': <DstTzInfo 'Europe/Paris' LMT+0:09:00 STD>,
 'updated': datetime.datetime(2018, 2, 15, 15, 19, 54, 834000, tzinfo=tzutc())}


The result is a JSON file, as in a normal request. If you want to have more interactive objects as response, you can use our special methods (there are explained in the next steps of the tutorial).

To help you get all relative paths in the API, you can use the ``Router`` object defined in ``radarly.utils.router``. This object stores, as attibutes, all the paths relative to a category.

In [12]:
from radarly.utils.router import Router
print('Routes relative to user'), pprint(Router.user)
print('\nRoutes relative to analytics details'), pprint(Router.analytics)
print('\nRoutes relative to publications'), pprint(Router.publication)

Routes relative to user
{'me': '/users.json'}

Routes relative to analytics details
{'global': '/projects/{project_id}/insights.json',
 'occupation': '/projects/{project_id}/insights/occupation.json'}

Routes relative to publications
{'metadata': '/projects/{project_id}/documents.json',
 'raw': '/projects/{project_id}/documents/raw.json',
 'search': '/projects/{project_id}/inbox/search.json'}


(None, None)

As you can see, most of the routes are just patterns and some path arguments must be specified (in most cases, the ``project_id``).

In [13]:
url = Router.publication['search'].format(project_id=2)
print('URL to search publication in the project 2:', url)
response = api.post(url, data={'platforms': ['instagram'], 'start': 0, 'limit': 15})
print('Response:')
pprint(response, depth=1)

URL to search publication in the project 2: /projects/2/inbox/search.json
Response:
{'hits': [...], 'stats': {}, 'took': 1546, 'total': 6568}


### Check the rates

Some limitations on the number of requests you can made to our API have been defined. You can be aware of these limitations by checking the ``rates`` attributes of the API. Each road of the API are not subjected to the same limits. There are three categories of rate limits: slow, medium and default (the routes in slow category are routes with the most restrictive limits). Please refer to the official documentation of the API for further informations.

In [14]:
api.rates

<RateLimit.slow:0.00%.medium:0.02%.default:0.01%>

You can then explore each category of rate limits.

In [15]:
api.rates.slow

{'limit': 0, 'remaining': 5000, 'reset': 0}

In [16]:
api.rates.medium

{'limit': 1, 'remaining': 4999, 'reset': 0}

In [17]:
api.rates.default

{'limit': 2, 'remaining': 14998, 'reset': 0}

You can check the category of an URL with the ``RateConf`` object, defined in ``radarly.rate``.

In [18]:
from radarly.rate import RateConf
url = '/users.json'
print(f'Category for the URL {url}:', RateConf.get_category(url))
url = '/projects/2/inbox/search.json'
print(f'Category for the URL {url}:', RateConf.get_category(url))

Category for the URL /users.json: default
Category for the URL /projects/2/inbox/search.json: medium


### Next step...

You know now how the ``RadarlyApi`` object works. In the next step of this tutorial, we will use other objects defined by the client in order to interact with the API.